In [198]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pycountry
from scipy.stats import norm
import random
from scipy.stats import norm
import re

# Dataset 2021

In [199]:
df2021=pd.read_csv('./Data/nlp_df2021.csv')
df2021.head()

,Unnamed: 0.1,Unnamed: 0,ResponseId,MainBranch,Employment,Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,OpSys,NEWStuck,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth
0,0,0,1,developer,"Independent contractor, freelancer, or self-em...",Slovakia,"Secondary school (e.g. American high school, G...",18 - 24 years,bootcampother,NaN,...,MacOS,overflowgo,No,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above
1,1,1,2,student,"Student, full-time",Netherlands,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,etcschool,7,...,Windows,stack,No,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above
2,2,2,3,developer,"Student, full-time",Russian Federation,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,forum,NaN,...,MacOS,itwatch,Yes,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above
3,3,3,4,developer,Employed full-time,Austria,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,other,NaN,...,Windows,overflowgo,No,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN
4,4,4,5,developer,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,friend,17,...,Linux-based,itwatch,No,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN


In [200]:
df2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0.1                  83439 non-null  int64  
 1   Unnamed: 0                    83439 non-null  int64  
 2   ResponseId                    83439 non-null  int64  
 3   MainBranch                    83439 non-null  object 
 4   Employment                    83323 non-null  object 
 5   Country                       83439 non-null  object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     83439 non-null  object 
 9   YearsCode                     81641 non-null  object 
 10  YearsCodePro                  61216 non-null  object 
 11  DevType                       66484 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

In [201]:
df2021.isnull().sum()

Unnamed: 0.1                        0
Unnamed: 0                          0
ResponseId                          0
MainBranch                          0
Employment                        116
Country                             0
EdLevel                           313
Age1stCode                        196
LearnCode                           0
YearsCode                        1798
YearsCodePro                    22223
DevType                         16955
OrgSize                         22713
CompTotal                       36256
CompFreq                        31289
LanguageHaveWorkedWith           1082
LanguageWantToWorkWith           6618
DatabaseHaveWorkedWith          13893
DatabaseWantToWorkWith          25140
PlatformHaveWorkedWith          31304
PlatformWantToWorkWith          41820
WebframeHaveWorkedWith          21732
WebframeWantToWorkWith          31344
MiscTechHaveWorkedWith          36384
MiscTechWantToWorkWith          45418
ToolsTechHaveWorkedWith         10902
ToolsTechWan

In [202]:
df2021=df2021.drop(['Unnamed: 0'], axis=1)#drop the unnecessary columns
df2021=df2021.drop(['Unnamed: 0.1'], axis=1)

## Functions for analysis

In [203]:
def sorted_value_counts(df, column):
    # Get the value counts for the specified column
    value_counts = df[column].value_counts()
    
    # Ensure the index is of type string to handle alphabetical sorting correctly
    value_counts.index = value_counts.index.astype(str)
    
    # Sort the value counts by the index alphabetically
    value_counts = value_counts.sort_index()
    
    # Return the sorted value counts
    return value_counts

In [204]:
def substr_replace(df, column,substr, replacement):
    df[column] = np.where(
    df[column].str.contains(substr, case=False, na=False, regex=False), 
    replacement, 
    df[column]
    )

In [205]:
def calculate_midpoint_age1stcode(age_col):
    if isinstance(age_col, str):
        if 'Younger than' in age_col:
            return math.ceil((0 + 5) / 2)  # Assume 0-5 years for 'Younger than 5 years'
        elif 'Older than' in age_col:
            return math.ceil(64 + (69 - 64) / 2)  # Assume 64-69 years for 'Older than 64 years'
        else:
            start, end = age_col.replace(' years', '').split(' - ')
            start = int(start)
            end = int(end)
            return  math.ceil((start + end) / 2)
    return None  # Handle non-string cases


In [206]:
def calculate_age_midpoint(age_col):
    if isinstance(age_col, str):
        if 'Under' in age_col:
            return math.ceil((0 + 18) / 2)  # Assume 0-18 years for 'Under 18 years'
        elif 'older' in age_col:
            return math.ceil(65 + (69 - 65) / 2)  # Assume 65-69 years for 'Older than 65 years'
        elif '-' in age_col:  # Check if the string contains a hyphen
            start, end = age_col.replace(' years old', '').split('-')
            start = int(start)
            end = int(end)
            return math.ceil((start + end) / 2)
        else:
            # If the string doesn't contain a hyphen, return None or handle accordingly
            return None
    return None  # Handle non-string cases

## Analysing columns

In [207]:
df2021['Employment'].value_counts()

Employment
Employed full-time                                      53584
Student, full-time                                      11781
Independent contractor, freelancer, or self-employed     8041
Not employed, but looking for work                       2961
Employed part-time                                       2461
Student, part-time                                       2051
Not employed, and not looking for work                   1228
I prefer not to say                                       890
Retired                                                   326
Name: count, dtype: int64

In [208]:
df2021['Employment'] = df2021['Employment'].fillna(method='ffill') #fill the missing values with the previous value

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\190981787.py:1: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [209]:
df2021['EdLevel']=df2021['EdLevel'].fillna("Something else")#fill the missing values with "Something else"
df2021['EdLevel'].value_counts()

EdLevel
Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          35357
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       17512
Some college/university study without earning a degree                                10589
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     9534
Other doctoral degree (Ph.D., Ed.D., etc.)                                             2567
Primary/elementary school                                                              2479
Associate degree (A.A., A.S., etc.)                                                    2231
Something else                                                                         1914
Professional degree (JD, MD, etc.)                                                     1256
Name: count, dtype: int64

In [210]:
df2021['Age1stCode_avg'] = df2021['Age1stCode'].apply(calculate_midpoint_age1stcode)
df2021['Age1stCode_avg']=df2021['Age1stCode_avg'].fillna(math.ceil(df2021['Age1stCode_avg'].mean()))#fill the missing values with the mean
df2021['Age1stCode_avg'].value_counts()

Age1stCode_avg
14.0    44170
21.0    20065
8.0     11720
30.0     4199
40.0     1312
3.0       905
50.0      544
60.0      245
17.0      196
67.0       83
Name: count, dtype: int64

In [211]:
df2021['Age_avg'] = df2021['Age'].apply(calculate_age_midpoint)
df2021['Age_avg']=df2021['Age_avg'].fillna(math.ceil(df2021['Age_avg'].mean()))#fill the missing values with the mean
df2021['Age_avg'].value_counts()

Age_avg
30.0    32568
21.0    20993
40.0    15183
50.0     5472
9.0      5376
60.0     1819
31.0     1607
67.0      421
Name: count, dtype: int64

In [212]:
df2021['LearnCode'].value_counts()

LearnCode
online                    21614
certification              8926
other                      8076
school                     7242
bootcampother              5835
forumonline                5198
family                     4044
forum                      3352
please                     2292
etcschool                  2292
friend                     2063
schoolonline               1807
etcschoolfriend            1764
certificationcolleague     1592
etcfriend                  1416
certificationother         1135
colleague                   683
schoolother                 673
forumother                  619
etcother                    599
bootcamp                    497
forumcolleague              383
bootcampschoolonline        327
etcschoolother              296
schoolcolleague             263
bootcamponline              215
forumfriend                 151
specifycolleague             85
Name: count, dtype: int64

In [213]:
df2021['YearsCode']=df2021['YearsCode'].fillna(method='ffill')#fill the missing values with the previous value
df2021['YearsCodePro']=df2021['YearsCodePro'].fillna(method='ffill')#fill the missing values with the previous value

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\81534222.py:1: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\81534222.py:2: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [214]:
df2021['OpSys']=df2021['OpSys'].fillna(method='ffill')#fill the missing values with the previous value
df2021["OpSys"].value_counts()

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\1991386150.py:1: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



OpSys
Windows                              37821
Linux-based                          21121
MacOS                                21025
Windows Subsystem for Linux (WSL)     2749
Other (please specify):                576
BSD                                    147
Name: count, dtype: int64

In [215]:
df2021['Sexuality'].value_counts()

Sexuality
Straight / Heterosexual                                                           61094
Prefer not to say                                                                  4783
Bisexual                                                                           2879
Gay or Lesbian                                                                     1367
Prefer to self-describe:                                                           1258
Straight / Heterosexual;Bisexual                                                    492
Queer                                                                               376
Bisexual;Queer                                                                      273
Straight / Heterosexual;Prefer to self-describe:                                    225
Gay or Lesbian;Queer                                                                149
Straight / Heterosexual;Queer                                                        78
Bisexual;Gay or Lesbia

In [216]:
df2021['DevType']=df2021['DevType'].fillna(method='ffill')#fill the missing values with "Something else"
df2021['DevType'].value_counts()

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\1383141237.py:1: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



DevType
Developer, full-stack                                                                                                                                                                                                                  10781
Developer, back-end                                                                                                                                                                                                                     6870
Developer, front-end                                                                                                                                                                                                                    2989
Developer, front-end;Developer, full-stack;Developer, back-end                                                                                                                                                                          2628
Developer, full-stack;Developer, back-end   

In [217]:
df2021['CompFreq']=df2021['CompFreq'].fillna(df2021['CompFreq'].mode()[0])#fill the missing values with the previous value
df2021['CompFreq'].isnull().sum()

0

In [218]:
df2021['CompTotal']=df2021['CompTotal'].fillna(df2021['CompTotal'].mean())#fill the missing values with the mean
df2021['CompTotal'].isnull().sum()

0

In [219]:
df2021 = df2021.drop(columns=['MiscTechHaveWorkedWith'])
df2021 = df2021.drop(columns=['MiscTechWantToWorkWith'])
df2021 = df2021.drop(columns=['PlatformWantToWorkWith'])
df2021 = df2021.drop(columns=['PlatformHaveWorkedWith'])
df2021=df2021.drop(columns=['Age'])
df2021=df2021.drop(columns=['Age1stCode'])
df2021=df2021.drop(columns=['Trans'])

In [220]:
df2021['LanguageHaveWorkedWith'].value_counts()[0:10]

LanguageHaveWorkedWith
Python                                    1337
HTML/CSS;JavaScript;Node.js;TypeScript     811
Java                                       715
HTML/CSS;JavaScript;PHP;SQL                674
C#                                         597
C#;HTML/CSS;JavaScript;SQL                 558
HTML/CSS;JavaScript                        531
HTML/CSS;JavaScript;Node.js                494
HTML/CSS;JavaScript;TypeScript             491
HTML/CSS;JavaScript;Python                 432
Name: count, dtype: int64

In [221]:
df2021['LanguageWantToWorkWith'].value_counts()

LanguageWantToWorkWith
Python                                                                             1389
Rust                                                                                821
HTML/CSS;JavaScript;Node.js;TypeScript                                              812
C#                                                                                  700
Java                                                                                512
                                                                                   ... 
C#;HTML/CSS;Java;JavaScript;Node.js;Objective-C;PHP;Python;SQL;Swift;TypeScript       1
C#;Elixir;Erlang;HTML/CSS;JavaScript;Python;Ruby;Rust;SQL                             1
C#;Go;JavaScript;Kotlin;Node.js;Python;SQL;Swift;TypeScript                           1
Bash/Shell;C;C++;HTML/CSS;Java;JavaScript;Objective-C;PHP;PowerShell;SQL              1
Elixir;HTML/CSS;Java;JavaScript;Node.js;PHP;SQL;TypeScript                            1
Name: cou

In [222]:
df2021['MentalHealth'].value_counts()

MentalHealth
None of the above                                                                                                                                                                                                                                    56459
Prefer not to say                                                                                                                                                                                                                                     3932
I have a concentration and/or memory disorder (e.g. ADHD)                                                                                                                                                                                             3059
I have an anxiety disorder                                                                                                                                                                                                                

In [223]:
df2021['Gender'].value_counts()

Gender
Man                                                                                   74817
Woman                                                                                  4120
Prefer not to say                                                                      1442
Non-binary, genderqueer, or gender non-conforming                                       690
Or, in your own words:                                                                  413
Man;Or, in your own words:                                                              268
Man;Non-binary, genderqueer, or gender non-conforming                                   252
Woman;Non-binary, genderqueer, or gender non-conforming                                 147
Man;Woman                                                                                41
Non-binary, genderqueer, or gender non-conforming;Or, in your own words:                 21
Man;Woman;Non-binary, genderqueer, or gender non-conforming              

In [224]:
df2021['LanguageHaveWorkedWith']=df2021['LanguageHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['LanguageWantToWorkWith']=df2021['LanguageWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['DatabaseHaveWorkedWith']=df2021['DatabaseHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['DatabaseWantToWorkWith']=df2021['DatabaseWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['WebframeHaveWorkedWith']=df2021['WebframeHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['WebframeWantToWorkWith']=df2021['WebframeWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['ToolsTechHaveWorkedWith']=df2021['ToolsTechHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['ToolsTechWantToWorkWith']=df2021['ToolsTechWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['NEWCollabToolsHaveWorkedWith']=df2021['NEWCollabToolsHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['NEWCollabToolsWantToWorkWith']=df2021['NEWCollabToolsWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2021['OrgSize']=df2021['OrgSize'].fillna(method='ffill')
df2021['Accessibility']=df2021['Accessibility'].fillna(method='ffill')
df2021['MentalHealth']=df2021['MentalHealth'].fillna(method='ffill')
df2021['NEWOtherComms']=df2021['NEWOtherComms'].fillna(method='ffill')
df2021['Ethnicity']=df2021['Ethnicity'].fillna(method='ffill')
df2021['Sexuality']=df2021['Sexuality'].fillna('Prefer not to say')
df2021['Gender']=df2021['Gender'].fillna("Prefer not to say")

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\4242856068.py:1: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\4242856068.py:2: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\4242856068.py:3: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\4242856068.py:4: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\4242856068.py:5: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfi

In [225]:
mean_comp_total = df2021.loc[df2021['CompTotal'] <= 1000000, 'CompTotal'].mean()

df2021.loc[df2021['CompTotal'] > 1000000, 'CompTotal'] = mean_comp_total

df2021.reset_index(drop=True, inplace=True)  # Reset index if needed

print(df2021['CompTotal'].describe())

count      83439.000000
mean      102961.667833
std       112840.529772
min            0.000000
25%        55000.000000
50%       102961.667833
75%       102961.667833
max      1000000.000000
Name: CompTotal, dtype: float64


In [226]:
#fill the missing values with the previous value
df2021['OrgSize'].value_counts()

OrgSize
20 to 99 employees                                    17054
100 to 499 employees                                  14365
10,000 or more employees                              11100
2 to 9 employees                                       9067
1,000 to 4,999 employees                               8590
10 to 19 employees                                     7465
Just me - I am a freelancer, sole proprietor, etc.     5679
500 to 999 employees                                   5184
5,000 to 9,999 employees                               3116
I don’t know                                           1819
Name: count, dtype: int64

In [227]:
df2021.dropna(inplace=True)

In [228]:
df2021.isnull().sum()

ResponseId                      0
MainBranch                      0
Employment                      0
Country                         0
EdLevel                         0
LearnCode                       0
YearsCode                       0
YearsCodePro                    0
DevType                         0
OrgSize                         0
CompTotal                       0
CompFreq                        0
LanguageHaveWorkedWith          0
LanguageWantToWorkWith          0
DatabaseHaveWorkedWith          0
DatabaseWantToWorkWith          0
WebframeHaveWorkedWith          0
WebframeWantToWorkWith          0
ToolsTechHaveWorkedWith         0
ToolsTechWantToWorkWith         0
NEWCollabToolsHaveWorkedWith    0
NEWCollabToolsWantToWorkWith    0
OpSys                           0
NEWStuck                        0
NEWOtherComms                   0
Gender                          0
Sexuality                       0
Ethnicity                       0
Accessibility                   0
MentalHealth  

## Visualizations Functions

In [229]:
def plot_boxplot(data, x, y, title):
    fig = go.Figure()
    for group_name, group_data in data.groupby(x):
        fig.add_trace(go.Box(y=group_data[y], name=group_name))
    fig.update_layout(title=title, xaxis_title=x, yaxis_title=y)
    st.plotly_chart(fig)

In [230]:
def plot_bar(data, x, y, title):
    fig = px.bar(data, x=x, y=y, title=title)
    return fig

In [231]:
def plot_bar_plotly(df, column_name, top_n=10, height=450, width=700):
    df_counts = df[column_name].value_counts().head(top_n).reset_index()
    df_counts.columns = [column_name, 'Count']
    
    fig = px.bar(df_counts, x=column_name, y='Count', 
                 labels={column_name: column_name, 'Count': 'Number of Developers'},
                 color=column_name, color_discrete_sequence=px.colors.qualitative.Pastel)
    
    fig.update_layout(xaxis_title=column_name, yaxis_title='Number of Developers')
    fig.update_layout(height=height, width=width)

    return fig


In [232]:
def plot_comparison_plotly(df, column1_name, column2_name, top_n=10, height=450, width=700):
    # Get the top values of column1_name
    df_top = df[column1_name].value_counts().head(top_n).index.tolist()
    
    # Filter the dataframe to include only rows where column1_name is in the top values
    df_filtered = df[df[column1_name].isin(df_top)]
    
    # Group by column1_name and get the mean (or any other statistic) of column2_name
    df_grouped = df_filtered.groupby(column1_name)[column2_name].mean().reset_index()
    
    # Sort by column1_name to maintain order in the plot
    df_grouped = df_grouped.sort_values(by=column1_name)
    
    # Plotting using Plotly Express
    fig = px.bar(df_grouped, x=column1_name, y=column2_name,
                 labels={column1_name: column1_name, column2_name: f'Average {column2_name}'},
                 color_discrete_sequence=px.colors.qualitative.Pastel)
    
    fig.update_layout(xaxis_title=column1_name, yaxis_title=f'Average {column2_name}')
    fig.update_layout(height=height, width=width)

    return fig


In [233]:
import plotly.graph_objects as go
import plotly.express as px

def plot_pie_plotly(df, column_name, top_n=10, height=600, width=600):
    participation_rate = df[column_name].value_counts().keys().tolist()[:top_n]
    count = df[column_name].value_counts().tolist()[:top_n]

    fig_pie = go.Figure(data=[go.Pie(labels=participation_rate, values=count, hole=.3)])

    fig_pie.update_layout(
        title='Top {} Distribution'.format(column_name),
        title_x=0.5,  # Center the title
        height=height,
        width=width,
        showlegend=True,
        legend=dict(
            x=1,
            y=0.5,
            traceorder='normal',
            font=dict(
                size=12,
            ),
        ),
    )

    # color differentiation and spacing of labels
    fig_pie.update_traces(
        marker=dict(
            colors=px.colors.qualitative.Plotly,
            line=dict(color='#000000', width=2)
        ),
        textinfo='percent+label',
        textfont_size=14,
        textposition='inside',
        hoverinfo='label+percent+name',
    )

    return fig_pie


In [234]:
def plot_value_counts_plotly(df, column_name):
    colors = ['lightseagreen', 'lightgreen', 'lightyellow', 'lightcoral', 'lightsalmon', 'lavender']

    counts = df[column_name].value_counts()
    fig = go.Figure(go.Bar(x=counts.index, y=counts.values, marker_color=random.choice(colors)))
    fig.update_layout(title=f'Value Counts for {column_name}', xaxis_title='Response', yaxis_title='Count')
    return fig

In [235]:
def gender_vs_top5countries(df):
    all_data = df.groupby(['Country', 'Gender']).size().reset_index(name='Count')
    all_data['Total'] = all_data.groupby('Country')['Count'].transform('sum')
    all_data['Percentage'] = all_data['Count'] / all_data['Total'] * 100


    top_countries = all_data.groupby('Country')['Total'].max().nlargest(5).index
    top_data = all_data[all_data['Country'].isin(top_countries)]

    # men and women data
    men_data = top_data[top_data['Gender'] == 'Man']
    women_data = top_data[top_data['Gender'] == 'Woman']

    fig = go.Figure()

    #bars for 'Men'
    fig.add_trace(go.Bar(x=men_data['Country'], y=men_data['Percentage'], name='Men', marker_color='darkblue'))

    #bars for 'Women'
    fig.add_trace(go.Bar(x=women_data['Country'], y=women_data['Percentage'], name='Women', marker_color='#5E96E9'))

    fig.update_layout(
        title='Gender vs Top 5 Countries in 2019',
        xaxis_title='Top 5 Countries',
        yaxis_title='Percentage',
        barmode='group'
    )

    return fig

In [236]:

def highest_paying_countries(df, salary_cap=1000000, top_n=10, name='Median'):
    # Filter data for Data Scientists
    ds = df[df['DevType'].str.contains('Data scientist', na=False)]
    
    # Remove rows with unrealistic salaries
    ds = ds[ds['CompTotal'] <= salary_cap]
    
    # Group by country and calculate median salary
    ds_median_salary = ds.groupby('Country')['CompTotal'].median().reset_index(name=name)
    
    # Sort values by median salary in descending order
    ds_median_salary.sort_values(by=[name], ascending=False, inplace=True)
    
    # Select top n countries with highest median salaries
    top_median_salary = ds_median_salary.head(top_n)
    
    # Create a horizontal bar plot with multicolor bars
    fig = px.bar(top_median_salary, x=name, y='Country', orientation='h',
                 labels={name: f'Average Salary in US$ ({name})', 'Country': 'Country'},
                 title=f'Top {top_n} highest paying data scientist countries',
                 color='Country',  # Color bars by Country for multicolor effect
                 color_discrete_sequence=px.colors.qualitative.Set3)  # Use qualitative color palette
    
    # Update layout for better readability
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, 
                      title={'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
    
    return fig


In [237]:

def clustered_graph(df, col1, col2):   
    top_10 = df[col2].value_counts().nlargest(10).index

    filtered_df = df[df[col2].isin(top_10)]

    sns.catplot(data=filtered_df, x=col2, hue=col1, kind="count", height=6, aspect=2)
    plt.title(f'Clustered Bar Chart of {col1} and {col2}')
    plt.xticks(rotation=45)
    plt.show()

In [238]:
def stacked_graph_for_correlation(df, col1, col2, height=600, width=800):
    # Create a frequency table
    freq_table = df.groupby([col1, col2]).size().reset_index(name='counts')
    
    fig = px.bar(freq_table, x=col2, y='counts', color=col1, 
                 title=f'Distribution of {col1} by {col2}',
                 labels={col1: col1, col2: col2, 'counts': 'Counts'},
                 height=height, width=width)
    
    fig.update_layout(
        title={'text': f'{col1} Distribution by {col2}', 'x': 0.5},
        xaxis_title=col2,
        yaxis_title='Counts',
        legend_title=col1,
        legend=dict(x=1, y=1)
    )
    
    return fig

In [239]:
def map_mental_health(value):
    if pd.isna(value):
        return "Other/Prefer not to say/Or, in your own words"
    if "concentration and/or memory disorder" in value:
        return "Concentration/Memory Disorder"
    elif "anxiety disorder" in value:
        return "Anxiety Disorder"
    elif "mood or emotional disorder" in value:
        return "Mood/Emotional Disorder"
    elif "autism / an autism spectrum disorder" in value:
        return "Autism Spectrum Disorder"
    elif ";" in value:  # Multiple disorders
        return "Multiple Disorders"
    elif "Prefer not to say" in value or "Or, in your own words:" in value or "None of the above" in value:
        return "Other/Prefer not to say/Or, in your own words"
    else:
        return "Other/Prefer not to say/Or, in your own words"

df2021['MentalHealth'] = df2021['MentalHealth'].apply(map_mental_health)

# print(df.head())

In [240]:
def map_ethnicity(value):
    if pd.isna(value):
        return "Other/Prefer not to say"
    
    categories = {
        "White or of European descent": ["White or of European descent"],
        "South Asian": ["South Asian"],
        "Hispanic or Latino/a/x": ["Hispanic or Latino/a/x", "Hispanic or Latino/a/x;Or, in your own words:"],
        "Southeast Asian": ["Southeast Asian"],
        "East Asian": ["East Asian"],
        "Black or of African descent": ["Black or of African descent"],
        "Middle Eastern": ["Middle Eastern"],
        "Indigenous": ["Indigenous", "Native American", "Pacific Islander", "Indigenous Australian"],
        "Multiracial/Biracial": ["Multiracial", "Biracial"],
        "Other/Prefer not to say": ["Prefer not to say", "Or, in your own words:", "I don't know"]
    }
    
    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in value:
                return category
    return "Other/Prefer not to say"

df2021['EthnicityCategory'] = df2021['Ethnicity'].apply(map_ethnicity)

In [271]:
def counts(df, column_name, year):
    language_counts = df[column_name].str.split(';', expand=True).stack().value_counts().to_frame(name=year)
    language_counts[column_name] = language_counts.index
    language_counts.reset_index(drop=True, inplace=True)
    language_counts = language_counts[[column_name, year]]
    return language_counts

def compare_column_and_plot(column):
    languagedesire_2021 = counts(df2021, column, '2021')
    languagedesire_2022 = counts(df2022, column, '2022')

    print("2021 Data:")
    print(languagedesire_2021.head())
    print("2022 Data:")
    print(languagedesire_2022.head())

    # Merge language counts for both years
    languagedesire_all = pd.merge(languagedesire_2021, languagedesire_2022, on=column, how='outer')
    
    print("Merged Data:")
    print(languagedesire_all.head())

    # Fill NaN values with 0 and convert counts to integers
    languagedesire_all.fillna(0, inplace=True)
    languagedesire_all['2021'] = languagedesire_all['2021'].astype(int)
    languagedesire_all['2022'] = languagedesire_all['2022'].astype(int)
    
    languagedesire_all.set_index(column, inplace=True)
    
    print("Filled and Converted Data:")
    print(languagedesire_all.head())

    languagedesire19_20 = languagedesire_all.div(languagedesire_all.sum())
    
    print("Percentage Data:")
    print(languagedesire19_20.head())

    fig = go.Figure()

    colors = ['#636EFA', '#EF553B']  # Define custom colors for the bars

    for i, col in enumerate(languagedesire19_20.columns):
        fig.add_trace(go.Bar(
            x=languagedesire19_20.index, 
            y=languagedesire19_20[col], 
            name=col,
            marker=dict(color=colors[i % len(colors)])  # Apply custom colors
        ))

    fig.update_layout(
        title='Comparison of Language Desires by Year',
        xaxis_title=column,
        yaxis_title='Percentages',
        font=dict(size=14),
        barmode='group',
        height=600,
        width=800
    )

    return fig 

In [273]:
def counts(df, column_name):
    # Split the entries in the column, stack them, and count occurrences
    language_counts = df[column_name].str.split(';', expand=True).stack().value_counts().to_frame(name='Count')
    language_counts.reset_index(inplace=True)
    language_counts.columns = ['Language', 'Count']
    return language_counts

def compare_language_columns_and_plot(df, column1, column2):
    # Get counts for both columns
    worked_with_counts = counts(df, column1)
    want_to_work_with_counts = counts(df, column2)

    # Merge the counts on the language column
    all_languages = pd.merge(worked_with_counts, want_to_work_with_counts, on='Language', how='outer')
    
    # Fill NaN values with 0 and convert counts to integers
    all_languages.fillna(0, inplace=True)
    all_languages['Count_x'] = all_languages['Count_x'].astype(int)
    all_languages['Count_y'] = all_languages['Count_y'].astype(int)
    
    all_languages.set_index('Language', inplace=True)
    
    fig = go.Figure()

    colors = ['#636EFA', '#EF553B']  # Define custom colors for the bars

    for i, col in enumerate(all_languages.columns):
        fig.add_trace(go.Bar(
            x=all_languages.index, 
            y=all_languages[col], 
            name=col.split('_')[1],  # Use column names to label the traces
            marker=dict(color=colors[i % len(colors)])  # Apply custom colors
        ))

    fig.update_layout(
        title='Comparison of Languages Worked With and Want to Work With',
        xaxis_title='Programming Languages',
        yaxis_title='Count',
        font=dict(size=14),
        barmode='stack',
        height=600,
        width=800
    )

    return fig


## Graphs

In [241]:
fig=plot_value_counts_plotly(df2021,'NEWStuck')
fig.show()

In [242]:
plot_comparison_plotly(df2021,'EdLevel','CompTotal',10,450,700)

In [243]:
plot_comparison_plotly(df2021,'DevType','CompTotal',10,450,700)

In [244]:
fig = highest_paying_countries(df2021, salary_cap=1000000, top_n=10, name='Median')
fig.show()

In [245]:
highest_paying_countries(df2021, salary_cap=1000000, top_n=10, name='Median')

In [246]:
plot_pie_plotly(df2021, 'OpSys', 10, 700, 800)

In [247]:
plot_bar_plotly(df2021, 'DevType', 10, 700, 800)

In [248]:
plot_bar_plotly(df2021, 'Employment', top_n=10, height=500, width=1000)

In [249]:
plot_bar_plotly(df2021, 'EdLevel', top_n=10, height=500, width=1000)

In [250]:
plot_bar_plotly(df2021, 'DevType', top_n=10, height=500, width=1000)

In [251]:
fig = stacked_graph_for_correlation(df2021, 'MentalHealth', 'Sexuality', 1400, 1000)
fig.show()

In [272]:
fig = compare_column_and_plot('LanguageWantToWorkWith')
fig.show()

2021 Data:
  LanguageWantToWorkWith   2021
0             JavaScript  40208
1                 Python  37970
2               HTML/CSS  31924
3             TypeScript  29173
4                    SQL  28958
2022 Data:
  LanguageWantToWorkWith   2022
0             JavaScript  33404
1                 Python  31021
2             TypeScript  27642
3               HTML/CSS  26914
4                    SQL  26304
Merged Data:
  LanguageWantToWorkWith     2021     2022
0                    APL    602.0    655.0
1               Assembly   3894.0   3219.0
2             Bash/Shell  15258.0  14019.0
3                      C  10522.0   8291.0
4                     C#  19529.0  16597.0
Filled and Converted Data:
                         2021   2022
LanguageWantToWorkWith              
APL                       602    655
Assembly                 3894   3219
Bash/Shell              15258  14019
C                       10522   8291
C#                      19529  16597
Percentage Data:
                    

In [274]:
fig = compare_language_columns_and_plot(df2021, 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith')
fig.show()

In [275]:
fig = compare_language_columns_and_plot(df2021, 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith')
fig.show()

In [276]:
fig = compare_language_columns_and_plot(df2021, 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith')
fig.show()

In [252]:
clustered_graph(df2021,'MentalHealth','Sexuality')

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\4068930218.py:9: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



# 2022 Dataset

In [253]:
df2022=pd.read_csv('./Data/nlp_df2022.csv')
df2022.head()

,Unnamed: 0.1,Unnamed: 0,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Knowledge_5,Knowledge_6,Knowledge_7,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech
0,0,0,1,other,NaN,other,NaN,NaN,other,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2,profession,"Employed, full-time",remote,Hobby;Contribute to open-source projects,NaN,other,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,3,code,"Employed, full-time",remote,Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",college,Technical documentation;Blogs;Programming Game...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,4,profession,"Employed, full-time",remote,I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",mediaschool,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,5,profession,"Employed, full-time",remote,Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",college,Technical documentation;Blogs;Stack Overflow;O...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Analysis

In [254]:
df2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73268 entries, 0 to 73267
Data columns (total 68 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0.1                    73268 non-null  int64  
 1   Unnamed: 0                      73268 non-null  int64  
 2   ResponseId                      73268 non-null  int64  
 3   MainBranch                      73268 non-null  object 
 4   Employment                      71709 non-null  object 
 5   RemoteWork                      73268 non-null  object 
 6   CodingActivities                58899 non-null  object 
 7   EdLevel                         71571 non-null  object 
 8   LearnCode                       73268 non-null  object 
 9   LearnCodeOnline                 50685 non-null  object 
 10  LearnCodeCoursesCert            29389 non-null  object 
 11  YearsCode                       71331 non-null  object 
 12  YearsCodePro                    

In [255]:
df2022 = df2022.drop(df2022.columns[51:67], axis=1)
df2022=df2022.drop(['Unnamed: 0'], axis=1)
df2022=df2022.drop(['Unnamed: 0.1'], axis=1)
df2022 = df2022.drop(columns=['MiscTechHaveWorkedWith'])
df2022 = df2022.drop(columns=['MiscTechWantToWorkWith'])
df2022 = df2022.drop(columns=['PlatformWantToWorkWith'])
df2022 = df2022.drop(columns=['PlatformHaveWorkedWith'])
df2022=df2022.drop(columns=['Trans'])
df2022=df2022.drop(columns=['VCHostingPersonal use'])
df2022=df2022.drop(columns=['VCHostingProfessional use'])
df2022=df2022.drop(columns=['LearnCodeCoursesCert'])
df2022=df2022.drop(columns=['OfficeStackAsyncHaveWorkedWith'])
df2022=df2022.drop(columns=['OfficeStackAsyncWantToWorkWith'])
df2022=df2022.drop(columns=['OfficeStackSyncHaveWorkedWith'])
df2022=df2022.drop(columns=['OfficeStackSyncWantToWorkWith'])

In [256]:
df2022.isnull().sum()

ResponseId                          0
MainBranch                          0
Employment                       1559
RemoteWork                          0
CodingActivities                14369
EdLevel                          1697
LearnCode                           0
LearnCodeOnline                 22583
YearsCode                        1937
YearsCodePro                    21435
DevType                         11966
OrgSize                         22229
PurchaseInfluence               22299
Country                          1497
CompTotal                       34846
CompFreq                        28843
LanguageHaveWorkedWith           2293
LanguageWantToWorkWith           6241
DatabaseHaveWorkedWith          13147
DatabaseWantToWorkWith          22254
WebframeHaveWorkedWith          19724
WebframeWantToWorkWith          27146
ToolsTechHaveWorkedWith         19097
ToolsTechWantToWorkWith         26702
NEWCollabToolsHaveWorkedWith     2921
NEWCollabToolsWantToWorkWith     9160
OpSysProfess

In [257]:
df2022['Employment'] = df2022['Employment'].fillna(method='ffill') #fill the missing values with the previous value
df2022['EdLevel']=df2022['EdLevel'].fillna("Something else")#fill the missing values with "Something else"
df2022['CodingActivities']=df2022['CodingActivities'].fillna(method="ffill")#fill the missing values with "Something else"
df2022['YearsCode']=df2022['YearsCode'].fillna(method='ffill')#fill the missing values with the previous value
df2022['YearsCodePro']=df2022['YearsCodePro'].fillna(method='ffill')#fill the missing values with the previous value
df2022['LearnCodeOnline']=df2022['LearnCodeOnline'].fillna(method='ffill')#fill the missing values with the previous value
df2022['Blockchain']=df2022['Blockchain'].fillna(method='ffill')#fill the missing values with the previous value
df2022['DevType']=df2022['DevType'].fillna(method='ffill')#fill the missing values with "Something else"
df2022['OrgSize']=df2022['OrgSize'].fillna(method='ffill')
df2022['Accessibility']=df2022['Accessibility'].fillna(method='ffill')
df2022['MentalHealth']=df2022['MentalHealth'].fillna(method='ffill')
df2022['Ethnicity']=df2022['Ethnicity'].fillna(method='ffill')
df2022["PurchaseInfluence"]=df2022["PurchaseInfluence"].fillna(method='ffill')
df2022['Sexuality']=df2022['Sexuality'].fillna('Prefer not to say')
df2022['Gender']=df2022['Gender'].fillna("Prefer not to say")
df2022['CompFreq']=df2022['CompFreq'].fillna(df2022['CompFreq'].mode()[0])#fill the missing values with the previous value
df2022['CompTotal']=df2022['CompTotal'].fillna(df2022['CompTotal'].mean())#fill the missing values with the mean
df2022['LanguageHaveWorkedWith']=df2022['LanguageHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['LanguageWantToWorkWith']=df2022['LanguageWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['DatabaseHaveWorkedWith']=df2022['DatabaseHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['DatabaseWantToWorkWith']=df2022['DatabaseWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['WebframeHaveWorkedWith']=df2022['WebframeHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['WebframeWantToWorkWith']=df2022['WebframeWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['NEWCollabToolsHaveWorkedWith']=df2022['NEWCollabToolsHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['NEWCollabToolsWantToWorkWith']=df2022['NEWCollabToolsWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['OpSysPersonal use']=df2022['OpSysPersonal use'].fillna(method='ffill')#fill the missing values with the previous value
df2022['OpSysProfessional use']=df2022['OpSysProfessional use'].fillna(method='ffill')#fill the missing values with the previous value
df2022['ToolsTechHaveWorkedWith']=df2022['ToolsTechHaveWorkedWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['ToolsTechWantToWorkWith']=df2022['ToolsTechWantToWorkWith'].fillna(method='ffill')#fill the missing values with the previous value
df2022['Country']=df2022['Country'].fillna(method='ffill')#fill the missing values with the previous value
df2022['ProfessionalTech']=df2022['ProfessionalTech'].fillna(method='ffill')#fill the missing values with the previous value
df2022['VersionControlSystem']=df2022['VersionControlSystem'].fillna(method='ffill')#fill the missing values with the previous value
df2022['VCInteraction']=df2022['VCInteraction'].fillna(method='ffill')#fill the missing values with the previous value

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\1557613256.py:1: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\1557613256.py:3: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\1557613256.py:4: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\1557613256.py:5: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Nikita\AppData\Local\Temp\ipykernel_17576\1557613256.py:6: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfi

In [258]:
df2022.dropna(inplace=True)

In [259]:
df2022.isnull().sum()

ResponseId                      0
MainBranch                      0
Employment                      0
RemoteWork                      0
CodingActivities                0
EdLevel                         0
LearnCode                       0
LearnCodeOnline                 0
YearsCode                       0
YearsCodePro                    0
DevType                         0
OrgSize                         0
PurchaseInfluence               0
Country                         0
CompTotal                       0
CompFreq                        0
LanguageHaveWorkedWith          0
LanguageWantToWorkWith          0
DatabaseHaveWorkedWith          0
DatabaseWantToWorkWith          0
WebframeHaveWorkedWith          0
WebframeWantToWorkWith          0
ToolsTechHaveWorkedWith         0
ToolsTechWantToWorkWith         0
NEWCollabToolsHaveWorkedWith    0
NEWCollabToolsWantToWorkWith    0
OpSysProfessional use           0
OpSysPersonal use               0
VersionControlSystem            0
VCInteraction 

In [260]:
df2022['Country'].value_counts()

Country
United States of America                                13314
India                                                    6507
Germany                                                  5364
United Kingdom of Great Britain and Northern Ireland     4142
Canada                                                   2455
                                                        ...  
Djibouti                                                    1
Seychelles                                                  1
Solomon Islands                                             1
Saint Kitts and Nevis                                       1
Gambia                                                      1
Name: count, Length: 180, dtype: int64

In [261]:
df2022['LearnCodeOnline'].value_counts()

LearnCodeOnline
Technical documentation;Blogs;Written Tutorials;Stack Overflow                                                                                                                                                        1025
Technical documentation;Blogs;Written Tutorials;Stack Overflow;Online forum                                                                                                                                            995
Technical documentation;Blogs;Stack Overflow                                                                                                                                                                           826
Technical documentation;Blogs;Written Tutorials;Stack Overflow;Online books;Online forum                                                                                                                               744
Technical documentation;Blogs;Written Tutorials;Stack Overflow;Online books                                 

In [262]:
df2022['Age_avg'] = df2022['Age'].apply(calculate_age_midpoint)
df2022['Age_avg']=df2022['Age_avg'].fillna(math.ceil(df2022['Age_avg'].mean()))#fill the missing values with the mean
df2022['Age_avg'].value_counts()

Age_avg
30.0    28109
21.0    16645
40.0    13987
50.0     5281
9.0      3865
60.0     1978
67.0      554
32.0      521
Name: count, dtype: int64

In [263]:
df2022['CodingActivities'].value_counts()

CodingActivities
Hobby                                                                                                                                  21761
I don’t code outside of work                                                                                                            8690
Hobby;Contribute to open-source projects                                                                                                7993
Hobby;Freelance/contract work                                                                                                           4293
Hobby;School or academic work                                                                                                           3642
                                                                                                                                       ...  
Contribute to open-source projects;Freelance/contract work;Other (please specify):;School or academic work                               

In [264]:
df2022['LanguageHaveWorkedWith'].value_counts()

LanguageHaveWorkedWith
HTML/CSS;JavaScript;TypeScript                                                                 1254
Python                                                                                          950
HTML/CSS;JavaScript                                                                             915
HTML/CSS;JavaScript;PHP;SQL                                                                     747
C#;HTML/CSS;JavaScript;SQL;TypeScript                                                           575
                                                                                               ... 
JavaScript;Kotlin;Rust                                                                            1
Bash/Shell;C#;Dart;Groovy;HTML/CSS;JavaScript;Kotlin;Objective-C;PHP;Python;Ruby;Rust;Swift       1
Bash/Shell;C#;Java;PowerShell;SQL;TypeScript                                                      1
Bash/Shell;Groovy;HTML/CSS;Java;JavaScript;Perl;Python;SQL                   

In [280]:
mean_comp_total = df2022.loc[df2022['CompTotal'] <= 1000000, 'CompTotal'].mean()

df2022.loc[df2022['CompTotal'] > 1000000, 'CompTotal'] = mean_comp_total

df2022.reset_index(drop=True, inplace=True)  # Reset index if needed

print(df2022['CompTotal'].describe())

count      70940.000000
mean      110641.766488
std       106062.230734
min            0.000000
25%        70000.000000
50%       110641.766488
75%       110641.766488
max      1000000.000000
Name: CompTotal, dtype: float64


## Graphs

In [279]:
plot_bar_plotly(df2022, 'DevType', 10, 700, 800)

In [281]:
plot_comparison_plotly(df2022,'EdLevel','CompTotal',10,450,700)

In [282]:
plot_comparison_plotly(df2022,'DevType','CompTotal',10,450,700)

In [283]:
fig = highest_paying_countries(df2022, salary_cap=1000000, top_n=10, name='Median')
fig.show()

In [284]:
plot_comparison_plotly(df2022,'LearnCode','CompTotal')

In [285]:
plot_bar_plotly(df2022,'Employment',10,700,800)

In [286]:
plot_bar_plotly(df2022,'EdLevel',10,700,800)

In [287]:
fig = compare_language_columns_and_plot(df2022, 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith')
fig.show()

In [289]:
fig = compare_language_columns_and_plot(df2022, 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith')
fig.show()

In [293]:
fig = compare_language_columns_and_plot(df2022, 'OpSysPersonal use', 'OpSysProfessional use')
fig.show()

In [295]:

df2022['MentalHealth'] = df2022['MentalHealth'].apply(map_mental_health)
df2022['EthnicityCategory'] = df2022['Ethnicity'].apply(map_ethnicity)
fig = stacked_graph_for_correlation(df2022, 'MentalHealth', 'Sexuality', 1400, 1000)
fig.show()